# Intuitive Algorithm Solver

### Configuration

In [14]:
FILEPATH = r'./../brightspace-locker/Data assignment parcel transport 2 Large.xlsx'
# FILEPATH = r'./../other/Data assignment parcel transport 2 very small Small.xlsx'

### Initialisations

In [15]:
from graph_visualiser import visualise_graph
from integer_linear_problem import ILP
from typing import Dict, List, Set
from tqdm import tqdm
import pandas as pd
import numpy as np
import copy
import json

ilp = ILP(FILEPATH)

### Solver

In [16]:
def powerset(s: Set) -> Set:
    bitmasks = [1 << i for i in range(len(s))] # 1 << i is basically equal to 2^i
    # the n-th digits in the binary representation of bit_selection dictates whether the n-th item is selected (1 = selected, 0 = not selected)
    for bit_selection in range(1, 1 << len(s)): # start with 1 to prevent empty set
        yield {el for el, bitmask in zip(s, bitmasks) if bit_selection & bitmask}

z_min = float('inf')

x = False # TODO: REMOVE
fixHubLength = False # TODO: REMOVE
with tqdm(total=(1 << len(ilp.N)) - 1, ncols=100) as pbar:
    for hubs in powerset(ilp.N):
        pbar.update(1)

        non_hubs = ilp.N - hubs

        H = dict()
        for node in ilp.N:
            H[node] = 1 if node in hubs else 0

        E = {node: dict() for node in ilp.N}
        
        # non-hub to hub / hub to non-hub edges
        for non_hub in non_hubs:
            cost_dict = {hub: ilp.c[non_hub][hub] for hub in hubs} # filter on hubs
            min_cost_hub = min(cost_dict, key=cost_dict.get)
            for hub in hubs:
                E[non_hub][hub] = 1 if hub == min_cost_hub else 0
                E[hub][non_hub] = 1 if hub == min_cost_hub else 0 # since E_ij = E_ji for all i =/= j

        # non-hub to non-hub edges (including itself)
        for non_hub1 in non_hubs:
            for non_hub2 in non_hubs:
                E[non_hub1][non_hub2] = 0

        # hub to hub edges (including itself)
        for hub1 in hubs:
            for hub2 in hubs:
                E[hub1][hub2] = 1

        if fixHubLength:
            if x:
                continue
            if len(hubs) == 6:# TODO: REMOVE
                x = True
        else:
            current_z = ilp.get_z(H, E)
            if current_z >= z_min:
                continue

            z_min = current_z
            # TODO: remove deepcopy and remove copy import
            hubs_min = copy.deepcopy(hubs)
            non_hubs_min = copy.deepcopy(non_hubs)
            H_min = copy.deepcopy(H)
            E_min = copy.deepcopy(E)

print('(Possibly Sub-optimal) Solution:')
print(f'z = {z_min}')
print(f'Hubs = {hubs_min}')
print(f'Non-hubs = {non_hubs_min}')
print(f'H = {json.dumps(H_min, indent=4)}')
print(f'E = {json.dumps(E_min, indent=4)}')
print(ilp)
visualise_graph(ilp.N, H_min, E_min, ilp.c, ilp.filepath)

100%|███████████████████████████████████████████████████████| 32767/32767 [1:36:11<00:00,  5.68it/s]


(Possibly Sub-optimal) Solution:
z = 139184
Hubs = {10, 3, 14}
Non-hubs = {1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15}
H = {
    "1": 0,
    "2": 0,
    "3": 1,
    "4": 0,
    "5": 0,
    "6": 0,
    "7": 0,
    "8": 0,
    "9": 0,
    "10": 1,
    "11": 0,
    "12": 0,
    "13": 0,
    "14": 1,
    "15": 0
}
E = {
    "1": {
        "10": 0,
        "3": 1,
        "14": 0,
        "1": 0,
        "2": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 0,
        "8": 0,
        "9": 0,
        "11": 0,
        "12": 0,
        "13": 0,
        "15": 0
    },
    "2": {
        "10": 0,
        "3": 1,
        "14": 0,
        "1": 0,
        "2": 0,
        "4": 0,
        "5": 0,
        "6": 0,
        "7": 0,
        "8": 0,
        "9": 0,
        "11": 0,
        "12": 0,
        "13": 0,
        "15": 0
    },
    "3": {
        "1": 1,
        "2": 1,
        "4": 1,
        "5": 0,
        "6": 0,
        "7": 0,
        "8": 0,
        "9": 0,
        "11": 0,
    